In [ ]:
import sys
import os
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.dates import DateFormatter
import numpy as np
sys.path.append(os.path.join(os.path.pardir, 'lesview'))
from lesview import *
from lesview.plots import plot_box_field

In [ ]:
casename = 'lsc_ymc22_sbl_bbl_v2'
runs = {'r1': '', 'r2': '_rf'}
ds_fld = {}
for rkey in runs.keys():
    datapath = os.path.join(os.path.pardir, 'tests', '{:s}{:s}'.format(casename, runs[rkey]))
    filepath = os.path.join(datapath, 'fields.jld2')
    ds_fld[rkey] = OceananigansDataVolume(filepath=filepath, fieldname=['w']).dataset

In [ ]:
figpath  = 'overview_{:s}'.format(casename)
os.makedirs(figpath, exist_ok=True)

In [ ]:
g = 9.81
H = 30
u10 = 8
cd = 1.25e-3
rhoa = 1.225
rhoo = 1026
tau = rhoa/rhoo*cd*u10*u10
ustar = np.sqrt(tau)

In [ ]:
ds = ds_fld['r1']
levels = np.linspace(-0.04, 0.04, 41)
plot_box_field(ds.data_vars['w'], view='both', itime=-2, cmap='RdBu_r', levels=levels)

In [ ]:
ds = ds_fld['r1']
levels = np.linspace(-4, 4, 41)
tstamps = ['2000-01-05T00:00:00', '2000-01-11T00:00:00']
abc = 'abcd'
tags = ['Aligned', 'Opposite']
tags2 = ['T1', 'T2']
fig, axarr = plt.subplots(2, 2, subplot_kw={'projection': '3d', 'computed_zorder': False})
fig.set_size_inches(10,7.5)

for i, rkey in enumerate(runs.keys()):
    ds = ds_fld[rkey]
    for j, tstmp in enumerate(tstamps):
        ax = axarr[j,i]
        da = ds.data_vars['w'].sel(time=slice(tstmp,tstmp))/ustar
        im = plot_box_field(da, ax=ax, view='both', itime=0, cmap='RdBu_r', levels=levels, add_colorbar=False, add_timestamp=False)
        ax.set_xticks([0, np.pi/2*H, np.pi*H, np.pi*H*1.5, np.pi*2*H])
        ax.set_xticklabels(['0', '$\pi/2$', '$\pi$', '$3\pi/2$', '$2\pi$'])
        ax.set_xlabel('$x/H$')
        ax.set_yticks([0, np.pi/2*H, np.pi*H, np.pi*H*1.5, np.pi*2*H])
        ax.set_yticklabels(['0', '$\pi/2$', '$\pi$', '$3\pi/2$', '$2\pi$'])
        ax.set_ylabel('$y/H$')
        ax.set_zticks([-H, -0.5*H, 0])
        ax.set_zticklabels(['$-1.0$', '$-0.5$', '$0.0$'])
        ax.set_zlabel('$z/H$')
        ax.text2D(0.1, 0.8, '({:s})'.format(abc[i*2+j]), transform=ax.transAxes, va='top', ha='left', fontsize=10)
        if j == 0:
            ax.text2D(0.75, 0.9, '{:s}'.format(tags[i]), transform=ax.transAxes, va='top', ha='left', fontsize=12)
        if i == 0:
            ax.text2D(0.02, 0.1, '{:s}'.format(tags2[j]), transform=ax.transAxes, va='top', ha='left', fontsize=12)
        

ax_inset = axarr[1,1].inset_axes([-0.35, 0.95, 0.5, 0.03])
cb = plt.colorbar(im, cax=ax_inset, orientation='horizontal', label='$w/u_*$')
plt.subplots_adjust(top=1, bottom=0.02, left=0.0, right=0.92, hspace=-0.01, wspace=0.12)
figname = os.path.join(figpath, 'snapshot-w')
fig.savefig(figname, dpi = 300, facecolor='w')